#### Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from bcb import sgs
import seaborn as sns
from plotnine import *
import statsmodels.api as sm
from statsmodels.tsa import x13
import os 
import matplotlib.dates as mdates

#### 1. Dados gerais - Desocupação e Rendimento real 

In [ ]:
#PNAD - Variação mensal
pnad = sgs.get({'pnad':24369},start = "2019-02-02")

import urllib.request
import zipfile

url = "https://www2.census.gov/software/x-13arima-seats/x13as/windows/program-archives/x13as_ascii-v1-1-b59.zip"
nome_arquivo_zip = "x13as_ascii-v1-1-b59.zip"

# Baixa o arquivo
urllib.request.urlretrieve(url, nome_arquivo_zip)

# Extrai o arquivo zipado
with zipfile.ZipFile(nome_arquivo_zip, 'r') as zip_ref:
    zip_ref.extractall()
    
# Define o caminho do programa como variável de ambiente
os.environ["X13PATH"] = "x13as"

# Ajusta sazonalmente a pnad
pnad_sa = (
    pnad
    .assign(pnad_sa = lambda x: x13.x13_arima_analysis(endog = x.pnad).seasadj)
    .reset_index()
)

pnad_sa = round((pnad_sa),2)

1.1 Gráfico - PNAD x Dessaz

In [ ]:
plt.figure(figsize=(12,8))

def percent_formatter(x, pos):
    return f'{x:.2f}%'

plt.rcParams['font.family'] = 'Roc Grotesk'

plt.title('Taxa de desocupação - PNADC', fontsize=14, fontweight='bold', pad=20)

plt.ylim(4,15)

plt.plot(pnad_sa['Date'], pnad_sa['pnad'], color='limegreen', linewidth=2,label='Desocupação (%)')
plt.plot(pnad_sa['Date'], pnad_sa['pnad_sa'], color='dimgrey', linewidth=2,linestyle='--',label='Ajuste sazonal')

plt.legend(loc='upper left', fontsize='12', bbox_to_anchor=(0.68, 0.58))

ax = plt.gca()

ax.yaxis.set_major_formatter(FuncFormatter(percent_formatter))

for spine in ax.spines.values():
    spine.set_visible(False)
ax.spines['left'].set_visible(False)

plt.show()

1.2 Criando função que extrai as séries

In [ ]:
#Função que servirá para pegar as séries

def get_series(codigo, date):
    df = sgs.get({codigo:codigo},start =date)

    return df

#### 2. População Economicamente Ativa - Categorias

In [ ]:
data = "2022-06-02"

componentes_pea = {
    'PIA': 24370,
    'Empregados no setor público e privado': 24371,
    'Empregados no setor público': 24372,  
    'Empregados por conta própria': 24373,
    'Setor privado - Com carteira': 24375,
    'Setor privado - Sem carteira': 24376,
    'Setor privado - Total': 24377,
    'Empregadores':24374,
    'População na força de trabalho': 24378,
    'População ocupada':24379
}

df_pea = pd.DataFrame()

for nome, codigo in componentes_pea.items():
    df_pea[nome] = get_series(codigo, data)

df_pea

2.1 Dessazonalizando PIA, PEA e PO

In [ ]:
data = "2021-02-02"

componentes_contingentes = {
    'PIA': 24370,
    'PEA': 24378,
    'PO':24379
}

df_contingentes = pd.DataFrame()

for nome, codigo in componentes_contingentes.items():
    df_contingentes[nome] = get_series(codigo, data)

df_contingentes

# Ajusta sazonalmente

contingentes_sa = pd.DataFrame(index=df_contingentes.index)  

for column in df_contingentes.columns:
    contingentes_sa[column] = x13.x13_arima_analysis(endog=df_contingentes[column]).seasadj

pd.set_option('display.float_format', lambda x: '%.2f' % x)

contingentes_sa = contingentes_sa/1000
print(contingentes_sa)

2.2 Taxa de participação (PEA/PIA)

In [ ]:
from matplotlib.ticker import FuncFormatter

def percent_formatter(x, pos):
    return f'{x:.2f}%'

particip = (contingentes_sa['PEA']/contingentes_sa['PIA'])*100

cores = ['green', 'dimgrey']

plt.figure(figsize=(12, 8))

plt.rcParams['font.family'] = 'Roc Grotesk'

plt.title('Taxa de participação - ajuste sazonal', fontsize=14, fontweight='bold',pad=20)

plt.plot(contingentes_sa.index, particip, color='limegreen', linewidth=2)

#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))
#plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))

ax = plt.gca()

ax.yaxis.set_major_formatter(FuncFormatter(percent_formatter))

for spine in ax.spines.values():
    spine.set_visible(False)
ax.spines['left'].set_visible(False)

plt.show()

2.3 Contingentes populacionais - com ajuste sazonal

In [ ]:
cores = ['green', 'dimgrey']

plt.figure(figsize=(12, 8))

plt.rcParams['font.family'] = 'Roc Grotesk'

plt.title('Contingentes populacionais - ajuste sazonal\n(milhões de pessoas)', fontsize=14,fontweight='bold',pad=20)

ax1 = plt.gca()

ax1.plot(contingentes_sa.index, contingentes_sa['PEA'], color='limegreen', linewidth=2, label='PEA (eixo esq.)')

ax2 = ax1.twinx()

ax2.plot(contingentes_sa.index, contingentes_sa['PO'], color='dimgrey', linewidth=4, linestyle='dotted', label='PO (eixo dir.)')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left', fontsize='12', bbox_to_anchor=(0.72, 0.38))
ax1.spines['top'].set_visible(False)
ax2.spines['top'].set_visible(False)

#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))
#plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=6))

plt.show()

2.4 Criando índices - Informalidade

In [ ]:
indices = pd.DataFrame()

indices['Informal'] = (df_pea['Setor privado - Sem carteira']/df_pea['Empregados no setor público e privado'])*100
indices['Formal'] = (df_pea['Setor privado - Com carteira']/df_pea['Empregados no setor público e privado'])*100

df_pea['Formais'] = (df_pea['Setor privado - Com carteira'] + df_pea['Empregados no setor público'] + df_pea['Empregadores'])

delta = pd.DataFrame()

delta['Formal'] = ((df_pea['Formais'].diff()/df_pea['Empregados no setor público e privado'])*100)
delta['Informal'] = ((df_pea['Setor privado - Sem carteira'].diff()/df_pea['Empregados no setor público e privado'])*100)
delta['Total'] = ((df_pea['Empregados no setor público e privado'].diff()/df_pea['Empregados no setor público e privado']))*100

delta = delta.reset_index()

delta

2.5 - Gráfico de contribuições (Formais x informais)

In [ ]:
import matplotlib.dates as mdates

plt.figure(figsize=(12, 8))

plt.rcParams['font.family'] = 'Roc Grotesk' 

plt.title('Contribuições para o crescimento do emprego (%)\nmês a mês', fontsize=14)

plt.ylim(-0.6,1.0)

# Plotando a linha do total
plt.plot(delta['Date'], delta['Total'], color='black', linewidth=2.5, label='Total')

# Usando a coluna 'Date' para a posição das barras
plt.bar(delta['Date'], delta['Formal'], color='green', label='Com carteira', width=20)
plt.bar(delta['Date'], delta['Informal'], color='lightgreen', label='Sem carteira',width=20,alpha=0.8)

# Configuração do eixo X
plt.axhline(0, color='gray', linewidth=0.8)   
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))

plt.legend(loc='upper left', fontsize='medium', bbox_to_anchor=(0.21, 0.84))

ax = plt.gca()
for spine in ax.spines.values():
    spine.set_visible(False)
ax.spines['left'].set_visible(True)

plt.show()

#### 3. Renda - Rendimento, massa salarial ...

In [ ]:
#Importando dados de rendimento e massa real

componentes_rendimento = {
    'Rendimento real habitual': 24382,
    'Rendimento real - Com carteira': 24383,
    'Rendimento real - Sem carteira': 24384,  
    'Rendimento real efetivo': 24381,
    'Massa real efetiva': 28544,
    'Massa real habitual': 28545,
}
df_rendimento = pd.DataFrame()

for nome, codigo in componentes_rendimento.items():
    df_rendimento[nome] = get_series(codigo, data)

#Importando dados de renda dispnível - já são deflacionados e dessaz, portanto não passam pelo ajuste sazonal

renda_disponivel = {
    'Renda disponível das famílias':29027,
    'Renda disponível das famílias - restrita':29026
}

df_renda_disponivel = pd.DataFrame()
for nome, codigo in renda_disponivel.items():
    df_renda_disponivel[nome] = get_series(codigo, data)


3.1 Ajuste sazonal - Massa real de rendimentos, e rendimento real médio (efetivo e habitual)

In [ ]:
# Ajusta sazonalmente

df_rendimento = df_rendimento.fillna(0)

rendimento_sa = pd.DataFrame(index=df_rendimento.index)  

for column in df_rendimento.columns:
    rendimento_sa[column] = x13.x13_arima_analysis(endog=df_rendimento[column]).seasadj

rendimento_sa

3.2 Criando deltas - YoY, MoM 

In [ ]:
def delta_mom(df):
    mom_df = df.pct_change()*100
    mom_df = mom_df.rename(columns=lambda col: f"{col} MoM")

    return mom_df

def delta_yoy(df):
    yoy_df = pd.DataFrame(index=df.index[12:], columns=df.columns)
    
    for coluna in df.columns:
        yoy_df[coluna] = ((df[coluna].iloc[12:].values / df[coluna].iloc[:-12].values) - 1) * 100
    
    yoy_df = yoy_df.rename(columns=lambda col: f"{col} YoY")

    return yoy_df

def delta_tot(df):
    tot_df = pd.DataFrame(index=df.index[4:], columns=df.columns)

    for coluna in df.columns:
        tot_df[coluna] = ((df[coluna].iloc[4:].values / df[coluna].iloc[:-4].values) - 1) * 100

    tot_df = tot_df.rename(columns=lambda col: f"{col} ToT")

    return tot_df


#Rendimento real médio e massa real 
rendimento_sa_mom = delta_mom(rendimento_sa)
rendimento_sa_yoy = delta_yoy(rendimento_sa)
rendimento_sa_tot = delta_tot(rendimento_sa)

#Renda disponível das famílias
df_renda_disponivel_mom = delta_mom(df_renda_disponivel)
df_renda_disponivel_yoy = delta_yoy(df_renda_disponivel)
df_renda_disponivel_tot = delta_yoy(df_renda_disponivel)

3.3 Criando função para igualar os tamanhos dos dataframes

In [ ]:
#Função de compatibilidade
def compat_size(df,df2):
    df = pd.concat([df,df2],axis=1)
    df = df.dropna()

    return df

#Compatibilizando tamanho dos dataframes desejados
renda_disponivel = compat_size(df_renda_disponivel,df_renda_disponivel_yoy)
massa_habitual = compat_size(rendimento_sa, rendimento_sa_tot)
rendimento = rendimento_sa.iloc[:-1]

3.4 Criando funções para dois tipos diferentes de gráfico

In [ ]:
def growth_chart(df, titulo, coluna, coluna2):
    plt.figure(figsize=(12, 8))
    plt.rcParams['font.family'] = 'Roc Grotesk'

    plt.title(titulo, fontsize=14)

    ax1 = plt.gca()
    
    if coluna not in df.columns:
        raise ValueError(f"A coluna '{coluna}' deve existir em ambos os DataFrames.")

    ax1.bar(df.index, df[coluna], color='lightgreen', width=20,alpha=0.7)  
    ax2 = ax1.twinx()
    ax2.plot(df.index, df[coluna2], color='black', linewidth=2.5)

    ax1.tick_params(axis='both', which='both', length=0)  
    ax2.tick_params(axis='both', which='both', length=0)  

    ax1.spines['top'].set_visible(False)
    ax2.spines['top'].set_visible(False)

    plt.show()

####################

def line_chart(df, titulo, coluna, coluna2):
    plt.figure(figsize=(12,8))

    plt.rcParams['font.family'] = 'Roc Grotesk'

    plt.title(titulo, fontsize=14, fontweight='bold', pad=10)

    plt.ylim()

    plt.plot(df.index, df[coluna], color='limegreen', linewidth=2,label=coluna)
    plt.plot(df.index, df[coluna2], color='indigo', linewidth=2,label=coluna2)

    z1 = np.polyfit(df.index.astype(int), df[coluna], 1)  # Regressão linear
    p1 = np.poly1d(z1)
    plt.plot(df.index, p1(df.index.astype(int)), color='limegreen', linestyle='--', linewidth=1)

    z2 = np.polyfit(df.index.astype(int), df[coluna2], 1)  # Regressão linear
    p2 = np.poly1d(z2)
    plt.plot(df.index, p2(df.index.astype(int)), color='indigo', linestyle='--', linewidth=1)

    plt.legend(loc='upper left', fontsize='12', bbox_to_anchor=(0.15, 0.88))

    ax = plt.gca()

    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.spines['left'].set_visible(False)

    plt.show()


3.5 Puxando os gráficos desejados

In [ ]:
#Estilo bar plot x line plot
growth_chart(renda_disponivel, 
             'Renda disponível das famílias - R$\n(deflacionado e com ajuste sazonal)', 
             'Renda disponível das famílias YoY', 'Renda disponível das famílias')

growth_chart(massa_habitual, 
             'Massa real de rendimento médio - habitual - R$\n(com ajuste sazonal)', 
             'Massa real habitual ToT', 'Massa real habitual')

####################

#Estilo line plot
line_chart(rendimento,'Rendimento real médio (R$)\nEfetivo x Habitual','Rendimento real habitual', 'Rendimento real efetivo')

#### 4. CAGED - Análise por decomposição de setores

4.1 CAGED - Consolidado

In [ ]:
#Série original
caged = sgs.get({'caged':28763},start = "2019-02-02")

caged = caged.diff()
caged = caged.fillna(0)

#Série dessazonalizada
caged_sa = (
    caged
    .assign(caged_sa = lambda x: x13.x13_arima_analysis(endog = x.caged).seasadj)
    .reset_index()
)
caged_sa

4.2 CAGED - Saldo total

In [ ]:
cores = ['green','dimgrey']

plt.figure(figsize=(12,8))

plt.rcParams['font.family'] = 'Roc Grotesk' 

plt.title('Caged - Saldo total', fontsize=14)
plt.ylim()

sns.set_style('white')  

#Plot da variação anual

sns.lineplot(data=caged_sa, x='Date', y='caged', color='limegreen',linewidth=2, label= 'CAGED')
sns.lineplot(data=caged_sa, x='Date', y='caged_sa', color='dimgrey',linewidth=4,linestyle='dotted',label='Dessaz')

plt.ylabel('',fontsize=12)
plt.xlabel('Data', fontsize = 12)
plt.legend(loc='upper left', fontsize='medium',bbox_to_anchor=(0.98,0.98))

plt.ticklabel_format(style='plain', axis='y')

sns.despine()

plt.show()

4.3 Agrupamento por setores da economia

In [ ]:
#Saldo por agrupamento de atividade econômica

componentes_caged = {
    'Agropecuária': 28764,
    'Indústrias de transformação': 28766,  
    'Construção': 28770,
    'Comércio': 28771,
    'Serviços': 28772
}

df_setores = pd.DataFrame()

for nome, codigo in componentes_caged.items():
    df_setores[nome] = get_series(codigo, data)

df_setores = df_setores.diff()
df_setores = df_setores.fillna(0)

df_setores

4.4 Retirando a sazonalidade dos componentes

In [ ]:
# Ajusta sazonalmente a pnad

setores_sa = pd.DataFrame(index=df_setores.index)  

for column in df_setores.columns:
    setores_sa[column] = x13.x13_arima_analysis(endog=df_setores[column]).seasadj

pd.set_option('display.float_format', lambda x: '%.f' % x)

setores_sa

4.5 Plot dos setores - Dessaz

In [ ]:
cores = ['green', 'lime','black','purple','blue','red','orange','gold','gray']

plt.figure(figsize=(12,8))

plt.rcParams['font.family'] = 'Roc Grotesk' 

plt.title('Caged - Setores (Dessaz)', fontsize=14)

plt.ylim()

sns.set_style('white')  

for i, coluna in enumerate(setores_sa.columns):
    sns.lineplot(data=setores_sa, x=setores_sa.index, y=coluna, color=cores[i], label=coluna)

plt.title('Caged - Setores (Dessaz)', fontsize=14)
plt.ylabel('',fontsize=12)
plt.xlabel('Data', fontsize = 12)
plt.legend(loc='upper left', fontsize='medium',bbox_to_anchor=(0.68,0.28))

sns.despine()

plt.show()

4.6 Recolhendo os últimos dois valores do CAGED para fazer o gráfico de barras

In [ ]:
setores_bar = df_setores.iloc[-1]
setores_bar_last = df_setores.iloc[-2]

4.7 Bar plot - últimos resultados

In [ ]:
nomes = [
    'Agropecuária',
    'Indústria de\ntransformação',
    'Construção',
    'Comércio',
    'Serviços'
]

plt.figure(figsize=(10, 6))

plt.rcParams['font.family'] = 'Roc Grotesk' 

bar_width = 0.35

x = np.arange(len(setores_bar))

bars = plt.bar(x, setores_bar.values, width=bar_width, color='limegreen', label='Último CAGED')

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2.0, 
        height + 0.2, 
        f'{int(height)}', 
        ha='center', 
        va='bottom',
        fontsize=9  # Ajuste o tamanho da fonte aqui
    )

plt.bar(x + 0.25, setores_bar_last.values, width=0.12, color='dimgrey', label='CAGED do mês anterior')

plt.title('CAGED - Análise dos setores', fontsize=16)
plt.ylabel('Saldo', fontsize=14)
plt.ylim(-10000,120000)

plt.xticks(x + 0.225, nomes, rotation=0, fontsize = 10)

plt.legend()

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.show()